In [9]:
class Access(object):

    def __getattr__(self, name):
        '''
        该方法定义了你试图访问一个不存在的属性时的行为。
        因此，重载该方法可以实现捕获错误拼写然后进行重定向, 
        或者对一些废弃的属性进行警告。
        '''
        print('__getattr__')
        return super(Access, self).__getattr__(name)

    def __setattr__(self, name, value):
        ''' 
        是实现封装的解决方案，它定义了你对属性进行赋值和修改操作时的行为。
        不管对象的某个属性是否存在,它都允许你为该属性进行赋值,因此你可以为属性的值进行自定义操作。
        有一点需要注意，实现_setattr_时要避免"无限递归"的错误，下面的代码示例中会提到。
        '''
        print('__setattr__')
        return super(Access, self).__setattr__(name, value)

    def __delattr__(self, name):
        print('__delattr__')
        return super(Access, self).__delattr__(name)

    def __getattribute__(self, name):
        #定义了你的属性被访问时的行为
        print('__getattribute__')
        return super(Access, self).__getattribute__(name)

access = Access()
access.attr1 = True  # __setattr__调用
#access.attr1  # 属性存在,只有__getattribute__调用
print(access.attr1)
try:
    access.attr2  # 属性不存在, 先调用__getattribute__, 后调用__getattr__
except AttributeError:
    pass
del access.attr1  # __delattr__调用

__setattr__
__getattribute__
True
__getattribute__
__getattr__
__delattr__


In [10]:
class Access2(object):
    a = []

ac = Access2()
ac.b = []
print(ac.b)
print(ac.c)

[]


AttributeError: 'Access2' object has no attribute 'c'

In [31]:
a = [1,2,3]
b_iter = iter(a)

In [43]:
a.reverse()
b = reversed(a)

In [38]:
try:
    print(next(b_iter))  # 错误用法
except StopIteration:
    print('end')

end


In [53]:
from copy import *

In [54]:
deepcopy(a)

[1, 2, 3]